This notebook uses the wowsims library to compare Feral Druid trinket values for p2/p3.

In [1]:
import json
import wowsims

In [2]:
p2_settings = json.load(open('data/p2_feral.json'))

# Delete the database because our library bundles it
del p2_settings['raid']['parties'][0]['players'][0]['database']

# Deep copy the settings
p3_settings = json.loads(json.dumps(p2_settings))

# Replace the item
p3_settings['raid']['parties'][0]['players'][0]['equipment']['items'] = json.load(open('data/p3_feral_items.json'))

We load settings for p2, delete the database, copy them for p3 and then overwrite the items.
After this process, `p2_settings` and `p3_settings` are ready to be used as input to the sim.

The trinkets to consider are listed below.

In [3]:
trinkets = [45931, 46038, 47131, 45609, 47115, 47734, 47946, 47948, 44253, 42987]
trinket_names = ['Mjolnir', 'Dark Matter', 'Verdict[H]', 'Comet', 'Verdict', 'Supremacy', 'Volatile', 'Victor[H]', 'DMC-Agi', 'DMC-Str']

The following code goes through the list of trinkets and pairs each one with each trinket later in the list.
Each of those trinket pairs is used to overwrite the top/bottom trinket items in the settings for both p2 and p3.

In [4]:
results = []
for i in range(len(trinkets)-1):
    for j in range(i+1, len(trinkets)):
        active_settings = p2_settings
        top = {'id': trinkets[i] }
        bot = {'id': trinkets[j] }
        active_settings['raid']['parties'][0]['players'][0]['equipment']['items'][12] = top
        active_settings['raid']['parties'][0]['players'][0]['equipment']['items'][13] = bot
        p2_result = wowsims.runSim(active_settings)
        active_settings = p3_settings
        top = {'id': trinkets[i] }
        bot = {'id': trinkets[j] }
        active_settings['raid']['parties'][0]['players'][0]['equipment']['items'][12] = top
        active_settings['raid']['parties'][0]['players'][0]['equipment']['items'][13] = bot
        p3_result = wowsims.runSim(active_settings)
        results.append((top['id'], bot['id'], p2_result['raidMetrics']['dps']['avg'], p3_result['raidMetrics']['dps']['avg']))
results.sort(key=lambda x: x[2], reverse=True)

The results from the previous operation are stored in the `results` list.
That list is sorted by the p2 dps value from the sim.

The sorted list is then modified to replace the trinket id with the name defined above.
The dps values are also rebased so that the value is their difference relative to the top combination.

In [5]:
import util
names = dict(zip(trinkets, trinket_names))
[('Trink1', 'Trink2', 'P2', 'P3')] + \
    [(names[r[0]], names[r[1]], r[2]-results[0][2], r[3]-results[0][3]) for r in results][:9]

Trink1,Trink2,P2,P3
Mjolnir,Verdict[H],0,0
Mjolnir,Verdict,-68,-72
Mjolnir,Dark Matter,-115,-190
Mjolnir,Comet,-130,-111
Verdict[H],Verdict,-154,-192
Mjolnir,DMC-Agi,-196,-215
Mjolnir,DMC-Str,-208,-221
Dark Matter,Verdict[H],-218,-291
Verdict[H],Comet,-220,-201
